In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

In [19]:
gt = pd.read_csv('data/features/seattle_ground_truth_labels.csv')

In [33]:
gt

,label_id,ground_truth
0,9,1
1,12,1
2,20,1
3,22,1
4,23,1
...,...,...
16575,233166,0
16576,233167,1
16577,233171,1
16578,233172,1


In [21]:
labels = pd.read_csv('data/processed-labels/.csv')

In [35]:
#merge the two dataframes inner
df = pd.merge(gt, labels, on='label_id', how='inner')
df

,label_id,ground_truth,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,geometry,clustered,count,false_positive,distance,tag_list,description,tag_count,way_type,intersection_distance
0,9,1,CurbRamp,2.0,3TO1sxavsQLIliltuPh5yw,3,299.323669,-17.540178,180.368912,1.460938,e4ff325c-309b-4ea6-8555-8f973e662362,47.598358,-122.306519,POINT (-122.3065185546875 47.598358154296875),1.0,6.0,0,12.359854,1.0,0.0,1.0,tertiary,40.679454
1,12,1,SurfaceProblem,3.0,2zMHlmqFF6V2SHqueb1fZA,3,20.665178,-14.486608,89.751045,-1.658913,e4ff325c-309b-4ea6-8555-8f973e662362,47.598400,-122.306679,POINT (-122.30667877197266 47.5984001159668),0.0,3.0,0,1.758375,1.0,0.0,1.0,-1,-1.000000
2,20,1,CurbRamp,1.0,lk95zcW0edw2sZCB1rMPTA,3,341.866058,-12.957589,106.566940,-2.300713,e4ff325c-309b-4ea6-8555-8f973e662362,47.598362,-122.309052,POINT (-122.30905151367188 47.59836196899414),1.0,6.0,0,5.536850,0.0,0.0,0.0,residential,13.214736
3,22,1,CurbRamp,1.0,hYEH3dM1ydGeOvU7l906_A,3,210.609375,-13.984375,255.282394,0.888550,e4ff325c-309b-4ea6-8555-8f973e662362,47.586857,-122.304230,POINT (-122.30422973632812 47.586856842041016),1.0,4.0,0,11.320579,0.0,0.0,0.0,residential,27.025511
4,23,1,SurfaceProblem,4.0,hYEH3dM1ydGeOvU7l906_A,2,308.345978,-34.910713,255.282394,0.888550,e4ff325c-309b-4ea6-8555-8f973e662362,47.586967,-122.304184,POINT (-122.30418395996094 47.58696746826172),0.0,4.0,0,15.889878,1.0,0.0,1.0,-1,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16575,233166,0,Obstacle,4.0,hJDiPHlOoVDAS8AF4ZQ2ow,1,276.312500,-22.812500,304.674866,0.043660,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.549702,-122.323402,POINT (-122.32340240478516 47.54970169067383),0.0,5.0,0,18.215611,1.0,0.0,2.0,-1,-1.000000
16576,233167,1,NoSidewalk,5.0,IEprRf1khSshvYgwnVkRVA,1,312.500000,-17.187500,307.725586,0.072340,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.549858,-122.323418,POINT (-122.32341766357422 47.54985809326172),0.0,3.0,0,27.443284,0.0,0.0,0.0,-1,-1.000000
16577,233171,1,NoCurbRamp,5.0,hFVojVcV0UAWHtVa9Nnk7Q,1,155.562500,-31.437500,137.767288,-0.617290,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.550358,-122.323029,POINT (-122.32302856445312 47.550357818603516),1.0,6.0,0,24.258974,0.0,0.0,0.0,residential,33.350587
16578,233172,1,NoCurbRamp,5.0,hFVojVcV0UAWHtVa9Nnk7Q,1,155.562500,-31.437500,137.767288,-0.617290,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.550392,-122.322937,POINT (-122.32293701171875 47.550392150878906),1.0,6.0,0,20.184373,0.0,0.0,0.0,residential,25.462447


In [36]:
#transform gt_geo to a geodataframe and set the crs
gt_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))
gt_geo.crs = {'init' :'epsg:4326'}


In [37]:
gt_geo

,label_id,ground_truth,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,geometry,clustered,count,false_positive,distance,tag_list,description,tag_count,way_type,intersection_distance
0,9,1,CurbRamp,2.0,3TO1sxavsQLIliltuPh5yw,3,299.323669,-17.540178,180.368912,1.460938,e4ff325c-309b-4ea6-8555-8f973e662362,47.598358,-122.306519,POINT (-122.30652 47.59836),1.0,6.0,0,12.359854,1.0,0.0,1.0,tertiary,40.679454
1,12,1,SurfaceProblem,3.0,2zMHlmqFF6V2SHqueb1fZA,3,20.665178,-14.486608,89.751045,-1.658913,e4ff325c-309b-4ea6-8555-8f973e662362,47.598400,-122.306679,POINT (-122.30668 47.59840),0.0,3.0,0,1.758375,1.0,0.0,1.0,-1,-1.000000
2,20,1,CurbRamp,1.0,lk95zcW0edw2sZCB1rMPTA,3,341.866058,-12.957589,106.566940,-2.300713,e4ff325c-309b-4ea6-8555-8f973e662362,47.598362,-122.309052,POINT (-122.30905 47.59836),1.0,6.0,0,5.536850,0.0,0.0,0.0,residential,13.214736
3,22,1,CurbRamp,1.0,hYEH3dM1ydGeOvU7l906_A,3,210.609375,-13.984375,255.282394,0.888550,e4ff325c-309b-4ea6-8555-8f973e662362,47.586857,-122.304230,POINT (-122.30423 47.58686),1.0,4.0,0,11.320579,0.0,0.0,0.0,residential,27.025511
4,23,1,SurfaceProblem,4.0,hYEH3dM1ydGeOvU7l906_A,2,308.345978,-34.910713,255.282394,0.888550,e4ff325c-309b-4ea6-8555-8f973e662362,47.586967,-122.304184,POINT (-122.30418 47.58697),0.0,4.0,0,15.889878,1.0,0.0,1.0,-1,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16575,233166,0,Obstacle,4.0,hJDiPHlOoVDAS8AF4ZQ2ow,1,276.312500,-22.812500,304.674866,0.043660,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.549702,-122.323402,POINT (-122.32340 47.54970),0.0,5.0,0,18.215611,1.0,0.0,2.0,-1,-1.000000
16576,233167,1,NoSidewalk,5.0,IEprRf1khSshvYgwnVkRVA,1,312.500000,-17.187500,307.725586,0.072340,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.549858,-122.323418,POINT (-122.32342 47.54986),0.0,3.0,0,27.443284,0.0,0.0,0.0,-1,-1.000000
16577,233171,1,NoCurbRamp,5.0,hFVojVcV0UAWHtVa9Nnk7Q,1,155.562500,-31.437500,137.767288,-0.617290,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.550358,-122.323029,POINT (-122.32303 47.55036),1.0,6.0,0,24.258974,0.0,0.0,0.0,residential,33.350587
16578,233172,1,NoCurbRamp,5.0,hFVojVcV0UAWHtVa9Nnk7Q,1,155.562500,-31.437500,137.767288,-0.617290,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,47.550392,-122.322937,POINT (-122.32294 47.55039),1.0,6.0,0,20.184373,0.0,0.0,0.0,residential,25.462447


In [40]:
#split the dataframe into two dataframes, one groud truth == 1 and one ground truth == 0
validated_correct = gt_geo[gt_geo['ground_truth'] == 1]
validated_incorrect = gt_geo[gt_geo['ground_truth'] == 0]


In [42]:
label_colors ={'Crosswalk':'#FABF1C',
               'CurbRamp':'#90C31F',
               'NoCurbRamp':'#E679B6',
               'NoSidewalk':'#BE87D8',
               'Obstacle' :'#78B0EA',
               'Occlusion':'#B3B3B3',
               'Other':'#B3B3B3',
               'Signal':'#63C0AB',
               'SurfaceProblem':'#F68D3E'}

In [44]:
def get_label_colors(geof):
    label_list = geof['label_type'].unique().tolist()
    label_list.sort()
    j = []
    for i in label_list:
        j.append(label_colors[i])
    return j

In [62]:
m = validated_incorrect.explore(
        tiles="CartoDb Positron",
        marker_type="circle",
        tooltip=["label_type",'label_id','severity'],
        popup=["label_type", 'label_id','severity'],
        color = "#000000",
        marker_kwds={"radius":1.5, "fill":False},
        style_kwds={"stroke":True,"fillOpacity":0.2,"weight": 2, "Opacity":0.2}
    )

In [64]:
interacitve_map = validated_correct.explore(
                    m = m,
                       tiles="CartoDb Positron",
                       marker_type="circle",
                       tooltip=["label_type",'label_id','severity'],
                        popup=["label_type", 'label_id','severity'],
                       cmap=get_label_colors(validated_correct),
                       column="label_type",
                       marker_kwds={"radius":1.5, "fill":True},
                       style_kwds={"stroke":True,"fillOpacity":0.6,"weight": 2},
                      )


In [65]:
interacitve_map.save('interactive-maps/seattle-ground-truth-labels.html')